In [47]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import glob
import skimage
from skimage import data
from skimage.feature import register_translation
from skimage.feature.register_translation import _upsampled_dft
from scipy.ndimage import fourier_shift
import re
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc


In [48]:
mpl.rcParams['figure.figsize'] = (6, 4)
mpl.rcParams['figure.dpi'] = 150

First we find the folder that holds the images and create a file list based on the files in that folder.

In [49]:
DATA_FOLDER = os.path.join('C:\\', 'Users', 'gt8mar', 'patientg')
print(DATA_FOLDER)

C:\Users\gt8mar\patientg


In [50]:
%cd C:\Users\gt8mar\patientg
!dir

C:\Users\gt8mar\patientg
 Volume in drive C is Windows
 Volume Serial Number is 5E54-8AA7

 Directory of C:\Users\gt8mar\patientg

02/19/2020  05:46 PM    <DIR>          .
02/19/2020  05:46 PM    <DIR>          ..
02/19/2020  05:39 PM     1,310,748,160 Basler_acA1300-200um__23253950__20200219_173912670.avi
02/19/2020  05:39 PM     1,310,748,160 Basler_acA1300-200um__23253950__20200219_173931269.avi
02/19/2020  05:40 PM     1,310,748,160 Basler_acA1300-200um__23253950__20200219_174015545.avi
02/19/2020  05:40 PM     1,310,748,160 Basler_acA1300-200um__23253950__20200219_174041147.avi
02/19/2020  05:41 PM     1,310,748,160 Basler_acA1300-200um__23253950__20200219_174101709.avi
02/19/2020  05:41 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174159417_1.tiff
02/19/2020  05:41 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174159417_10.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174159417_100.tiff
02/19/2020  05:42 PM  

02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174230539_645.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174230539_646.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174230539_647.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174230539_648.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174230539_649.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174230539_65.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174230539_650.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174230539_651.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174230539_652.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174230539_653.tiff
02/19/2020 

02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174245878_200.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174245878_201.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174245878_202.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174245878_203.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174245878_204.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174245878_205.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174245878_206.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174245878_207.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174245878_208.tiff
02/19/2020  05:42 PM         1,319,058 Basler_acA1300-200um__23253950__20200219_174245878_209.tiff
02/19/2020

In [51]:
pwd


'C:\\Users\\gt8mar\\patientg'

These are the different video file collection in the folder patientg

174159417
174230539
174245878


In [52]:
%cd C:\Users\gt8mar\Documents\Marcus\Leuk

C:\Users\gt8mar\Documents\Marcus\Leuk


In [70]:
file_list = glob.glob(os.path.join(DATA_FOLDER, 'Basler_acA1300-200um__23253950__*.tiff'))
print(file_list)

['C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_1.tiff', 'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_10.tiff', 'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_100.tiff', 'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_1000.tiff', 'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_101.tiff', 'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_102.tiff', 'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_103.tiff', 'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_104.tiff', 'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_105.tiff', 'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_106.tiff', 'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_

In [78]:
file_list[5]

'C:\\Users\\gt8mar\\patientg\\Basler_acA1300-200um__23253950__20200219_174159417_102.tiff'

Next we create a list of numbers (num) that correspond to the file names using the regular expression package. We do this because the files will otherwise sort alphabetically, not numerically. We plan to make a dictionary instead. 

In [96]:
findall = re.compile('Basler_acA1300-200um__23253950__(?P<date>[\d]+)_(?P<coll>[\d]+)_(?P<num>[\d]+).tiff')


In [106]:
print(findall.search(file_list[5])['num'])
print(findall.search(file_list[5])['date'])
print(findall.search(file_list[5])['coll'])


102
20200219
174159417


Here we make a dictionary that has values of our numpy image arrays with keys of the file number. I will later add the date and run number to the dictionary. 

In [8]:
my_images = {}
for i in range(len(file_list)):
    #print(file_list[i])
    num = myre.search(file_list[i])['num']
    my_images[int(num)] = skimage.external.tifffile.imread(file_list[i])
    

In [9]:
sorted_myimages = sorted(my_images)
#print(sorted_myimages)
# print(max(sorted_myimages))



In [10]:
#print(my_images)

Now we have a dictionary where we can call our images by their file number. Next we will check to see how close each file is to it's neighbor. 

In [11]:
shift, error, diffphase = register_translation(my_images[999], my_images[1000], 100)
print(f"Detected subpixel offset (y, x): {shift}")

print(shift[0], shift[1])

Detected subpixel offset (y, x): [-0.01  0.  ]
-0.01 0.0


# This function shows us the offset between frames:

In [12]:




print(len(sorted_myimages))
print(sorted_myimages)
print(range(0, len(sorted_myimages), 1))
[(sorted_myimages[i], sorted_myimages[i+1]) for i in range(0, len(sorted_myimages)-1)]


#     shift, error, diffphase = register_translation(my_images[first], my_images[first+1], 100)
#     if shift[0]>0.5:
#         print(f"Discontinuity between {my_images[first]} and {my_images[first+1]}")
#         break
#     else:
#         print(first, first+1)
#         print(f"Detected subpixel offset (y, x): {shift}")
#         first += 1

    



first file = 991

11
[1, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]
range(0, 11)


[(1, 991),
 (991, 992),
 (992, 993),
 (993, 994),
 (994, 995),
 (995, 996),
 (996, 997),
 (997, 998),
 (998, 999),
 (999, 1000)]

In [14]:
print(len(sorted_myimages))
#print(sorted_myimages)
print(range(0, len(sorted_myimages), 1))
[(sorted_myimages[i], sorted_myimages[i+1]) for i in range(0, len(sorted_myimages)-1)]

1000
range(0, 1000)


[(1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10),
 (10, 11),
 (11, 12),
 (12, 13),
 (13, 14),
 (14, 15),
 (15, 16),
 (16, 17),
 (17, 18),
 (18, 19),
 (19, 20),
 (20, 21),
 (21, 22),
 (22, 23),
 (23, 24),
 (24, 25),
 (25, 26),
 (26, 27),
 (27, 28),
 (28, 29),
 (29, 30),
 (30, 31),
 (31, 32),
 (32, 33),
 (33, 34),
 (34, 35),
 (35, 36),
 (36, 37),
 (37, 38),
 (38, 39),
 (39, 40),
 (40, 41),
 (41, 42),
 (42, 43),
 (43, 44),
 (44, 45),
 (45, 46),
 (46, 47),
 (47, 48),
 (48, 49),
 (49, 50),
 (50, 51),
 (51, 52),
 (52, 53),
 (53, 54),
 (54, 55),
 (55, 56),
 (56, 57),
 (57, 58),
 (58, 59),
 (59, 60),
 (60, 61),
 (61, 62),
 (62, 63),
 (63, 64),
 (64, 65),
 (65, 66),
 (66, 67),
 (67, 68),
 (68, 69),
 (69, 70),
 (70, 71),
 (71, 72),
 (72, 73),
 (73, 74),
 (74, 75),
 (75, 76),
 (76, 77),
 (77, 78),
 (78, 79),
 (79, 80),
 (80, 81),
 (81, 82),
 (82, 83),
 (83, 84),
 (84, 85),
 (85, 86),
 (86, 87),
 (87, 88),
 (88, 89),
 (89, 90),
 (90, 91),
 (91, 92),
 (92, 93),
 (93,

# This function corrects the offset:

In [17]:
from itertools import product
key_combos = list(product([sorted_myimages[999]], sorted_myimages[:999]))
print(key_combos[70])

# for i, j in key_combos:
#     print(i, j)


(1000, 71)


In [13]:
print(sorted_myimages[:100])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


In [20]:
# Make a new dictionary for the stabilized video files

my_images_new = {}

# Make a list of tuples that match each subsequent file with the first file

from itertools import product
key_combos = list(product([sorted_myimages[999]], sorted_myimages[:999]))
#print(key_combos)

# Iterate through each file and correct it to the base file

for i, j in key_combos:
    shift, error, diffphase = register_translation(my_images[i], my_images[j], 100)
    if shift[0]>100:
        print(f"Discontinuity between {my_images[i]} and {my_images[j]} in y")
        break
    if shift[1]>100:
        print(f"Discontinuity between {my_images[i]} and {my_images[j]} in x")
        break
    else:
        print(i,j)
        print(f"Detected subpixel offset (y, x): {shift}")
        
        # Take the fourier transform of the file, shift it, and then transform it back.
        
        file_shift = shift
        file_offset = fourier_shift(np.fft.fftn(my_images[j]), file_shift)
        file_offset = np.fft.ifftn(file_offset)
        
        # Create new entries in the stabilized video library
        my_images_new[i] = my_images[i] 
        my_images_new[j] = file_offset.real.astype(int)
        
        shift, error, diffphase = register_translation(my_images_new[i], my_images_new[j], 100)
        print(f"Detected subpixel offset after adjustment (y, x): {shift}")
        print()
        print('------------------')
        print()
       

1000 1
Detected subpixel offset (y, x): [29.01 50.11]
Detected subpixel offset after adjustment (y, x): [0.   0.03]

------------------

1000 2
Detected subpixel offset (y, x): [29.91 50.91]
Detected subpixel offset after adjustment (y, x): [-0.07  0.03]

------------------

1000 3
Detected subpixel offset (y, x): [30.95 49.11]
Detected subpixel offset after adjustment (y, x): [-0.04 -0.05]

------------------

1000 4
Detected subpixel offset (y, x): [31.89 48.68]
Detected subpixel offset after adjustment (y, x): [-0.08 -0.01]

------------------

1000 5
Detected subpixel offset (y, x): [28.99 48.69]
Detected subpixel offset after adjustment (y, x): [-0.02  0.01]

------------------

1000 6
Detected subpixel offset (y, x): [28.08 49.12]
Detected subpixel offset after adjustment (y, x): [0.06 0.  ]

------------------

1000 7
Detected subpixel offset (y, x): [30.97 47.66]
Detected subpixel offset after adjustment (y, x): [-0.02 -0.04]

------------------

1000 8
Detected subpixel offset

Detected subpixel offset after adjustment (y, x): [-0.01 -0.05]

------------------

1000 61
Detected subpixel offset (y, x): [31.12 48.28]
Detected subpixel offset after adjustment (y, x): [0.1  0.02]

------------------

1000 62
Detected subpixel offset (y, x): [28.02 48.78]
Detected subpixel offset after adjustment (y, x): [0.03 0.02]

------------------

1000 63
Detected subpixel offset (y, x): [31.08 48.24]
Detected subpixel offset after adjustment (y, x): [0.11 0.19]

------------------

1000 64
Detected subpixel offset (y, x): [30.04 48.11]
Detected subpixel offset after adjustment (y, x): [ 0.03 -0.02]

------------------

1000 65
Detected subpixel offset (y, x): [29.03 49.35]
Detected subpixel offset after adjustment (y, x): [ 0.05 -0.57]

------------------

1000 66
Detected subpixel offset (y, x): [30.04 49.84]
Detected subpixel offset after adjustment (y, x): [ 0.02 -0.01]

------------------

1000 67
Detected subpixel offset (y, x): [28.99 49.84]
Detected subpixel offset a

Detected subpixel offset after adjustment (y, x): [-0.08  0.02]

------------------

1000 120
Detected subpixel offset (y, x): [30.18 43.98]
Detected subpixel offset after adjustment (y, x): [0.11 0.03]

------------------

1000 121
Detected subpixel offset (y, x): [32.99 44.11]
Detected subpixel offset after adjustment (y, x): [-0.02 -0.03]

------------------

1000 122
Detected subpixel offset (y, x): [29.97 44.41]
Detected subpixel offset after adjustment (y, x): [-0.03  0.03]

------------------

1000 123
Detected subpixel offset (y, x): [30.89 44.85]
Detected subpixel offset after adjustment (y, x): [-0.1   0.02]

------------------

1000 124
Detected subpixel offset (y, x): [29.89 44.17]
Detected subpixel offset after adjustment (y, x): [-0.08  0.04]

------------------

1000 125
Detected subpixel offset (y, x): [32.96 45.02]
Detected subpixel offset after adjustment (y, x): [-0.02 -0.03]

------------------

1000 126
Detected subpixel offset (y, x): [32.08 43.42]
Detected subpix

Detected subpixel offset after adjustment (y, x): [ 0.03 -0.09]

------------------

1000 179
Detected subpixel offset (y, x): [10.97 36.4 ]
Detected subpixel offset after adjustment (y, x): [-0.03 -0.01]

------------------

1000 180
Detected subpixel offset (y, x): [ 9.98 35.35]
Detected subpixel offset after adjustment (y, x): [-0.13  1.99]

------------------

1000 181
Detected subpixel offset (y, x): [10.94 35.23]
Detected subpixel offset after adjustment (y, x): [-0.04  0.03]

------------------

1000 182
Detected subpixel offset (y, x): [11.94 36.54]
Detected subpixel offset after adjustment (y, x): [-0.03 -0.05]

------------------

1000 183
Detected subpixel offset (y, x): [10.98 36.64]
Detected subpixel offset after adjustment (y, x): [-0.01 -0.02]

------------------

1000 184
Detected subpixel offset (y, x): [10.14 34.9 ]
Detected subpixel offset after adjustment (y, x): [ 0.08 -0.03]

------------------

1000 185
Detected subpixel offset (y, x): [12.91 36.81]
Detected subp

Detected subpixel offset after adjustment (y, x): [-0.04 -0.03]

------------------

1000 238
Detected subpixel offset (y, x): [ 0.11 30.47]
Detected subpixel offset after adjustment (y, x): [0.07 0.06]

------------------

1000 239
Detected subpixel offset (y, x): [ 0.99 29.44]
Detected subpixel offset after adjustment (y, x): [-0.03 -0.14]

------------------

1000 240
Detected subpixel offset (y, x): [3.000e-02 3.035e+01]
Detected subpixel offset after adjustment (y, x): [0.01 0.04]

------------------

1000 241
Detected subpixel offset (y, x): [ 0.89 29.06]
Detected subpixel offset after adjustment (y, x): [-0.08  0.01]

------------------

1000 242
Detected subpixel offset (y, x): [ 2.04 29.39]
Detected subpixel offset after adjustment (y, x): [ 0.02 -0.04]

------------------

1000 243
Detected subpixel offset (y, x): [ 0.86 30.64]
Detected subpixel offset after adjustment (y, x): [-0.11  0.01]

------------------

1000 244
Detected subpixel offset (y, x): [ 0.06 28.59]
Detected 

...

Detected subpixel offset after adjustment (y, x): [0.02 0.03]

------------------

1000 707
Detected subpixel offset (y, x): [-0.55 18.52]
Detected subpixel offset after adjustment (y, x): [ 0.39 -0.01]

------------------

1000 708
Detected subpixel offset (y, x): [ 0.03 16.52]
Detected subpixel offset after adjustment (y, x): [0.02 0.02]

------------------

1000 709
Detected subpixel offset (y, x): [-0.26 18.44]
Detected subpixel offset after adjustment (y, x): [0.49 0.11]

------------------

1000 710
Detected subpixel offset (y, x): [-0.04 18.5 ]
Detected subpixel offset after adjustment (y, x): [-0.01  0.02]

------------------

1000 711
Detected subpixel offset (y, x): [-0.81 16.97]
Detected subpixel offset after adjustment (y, x): [ 0.18 -0.13]

------------------

1000 712
Detected subpixel offset (y, x): [ 0.06 18.52]
Detected subpixel offset after adjustment (y, x): [ 0.   -2.04]

------------------

1000 713
Detected subpixel offset (y, x): [-0.16 18.4 ]
Detected subpixel o

Detected subpixel offset after adjustment (y, x): [-0.51 -1.75]

------------------

1000 766
Detected subpixel offset (y, x): [-0.02 16.35]
Detected subpixel offset after adjustment (y, x): [ 0.02 -1.82]

------------------

1000 767
Detected subpixel offset (y, x): [-0.65 14.72]
Detected subpixel offset after adjustment (y, x): [ 0.31 -0.07]

------------------

1000 768
Detected subpixel offset (y, x): [-0.02 14.61]
Detected subpixel offset after adjustment (y, x): [-0.01  0.03]

------------------

1000 769
Detected subpixel offset (y, x): [-0.58 16.12]
Detected subpixel offset after adjustment (y, x): [0.3  0.05]

------------------

1000 770
Detected subpixel offset (y, x): [-0.09 14.52]
Detected subpixel offset after adjustment (y, x): [-0.03  0.03]

------------------

1000 771
Detected subpixel offset (y, x): [ 0.67 14.53]
Detected subpixel offset after adjustment (y, x): [-0.32  0.  ]

------------------

1000 772
Detected subpixel offset (y, x): [ 0.03 16.39]
Detected subpix

1000 824
Detected subpixel offset (y, x): [0.   0.04]
Detected subpixel offset after adjustment (y, x): [ 0.   -0.02]

------------------

1000 825
Detected subpixel offset (y, x): [0.01 0.04]
Detected subpixel offset after adjustment (y, x): [-0.01 -0.02]

------------------

1000 826
Detected subpixel offset (y, x): [0.   0.03]
Detected subpixel offset after adjustment (y, x): [ 0.   -0.03]

------------------

1000 827
Detected subpixel offset (y, x): [0.   0.04]
Detected subpixel offset after adjustment (y, x): [ 0.   -0.02]

------------------

1000 828
Detected subpixel offset (y, x): [0.01 0.03]
Detected subpixel offset after adjustment (y, x): [-0.01 -0.03]

------------------

1000 829
Detected subpixel offset (y, x): [0.   0.03]
Detected subpixel offset after adjustment (y, x): [ 0.   -0.03]

------------------

1000 830
Detected subpixel offset (y, x): [0.01 0.03]
Detected subpixel offset after adjustment (y, x): [-0.01 -0.03]

------------------

1000 831
Detected subpixel 

Detected subpixel offset after adjustment (y, x): [-0.02 -0.03]

------------------

1000 884
Detected subpixel offset (y, x): [0.   0.02]
Detected subpixel offset after adjustment (y, x): [ 0.   -0.04]

------------------

1000 885
Detected subpixel offset (y, x): [0.01 0.02]
Detected subpixel offset after adjustment (y, x): [-0.02 -0.03]

------------------

1000 886
Detected subpixel offset (y, x): [0.01 0.02]
Detected subpixel offset after adjustment (y, x): [-0.02 -0.03]

------------------

1000 887
Detected subpixel offset (y, x): [0.01 0.03]
Detected subpixel offset after adjustment (y, x): [-0.01 -0.03]

------------------

1000 888
Detected subpixel offset (y, x): [0.01 0.02]
Detected subpixel offset after adjustment (y, x): [-0.01 -0.04]

------------------

1000 889
Detected subpixel offset (y, x): [0.01 0.02]
Detected subpixel offset after adjustment (y, x): [-0.02 -0.03]

------------------

1000 890
Detected subpixel offset (y, x): [0.01 0.02]
Detected subpixel offset af

Detected subpixel offset after adjustment (y, x): [0.   0.05]

------------------

1000 944
Detected subpixel offset (y, x): [0. 0.]
Detected subpixel offset after adjustment (y, x): [0. 0.]

------------------

1000 945
Detected subpixel offset (y, x): [ 0.   -0.02]
Detected subpixel offset after adjustment (y, x): [0.   0.04]

------------------

1000 946
Detected subpixel offset (y, x): [ 0.   -0.01]
Detected subpixel offset after adjustment (y, x): [0.   0.05]

------------------

1000 947
Detected subpixel offset (y, x): [ 0.   -0.01]
Detected subpixel offset after adjustment (y, x): [0.   0.05]

------------------

1000 948
Detected subpixel offset (y, x): [ 0.   -0.01]
Detected subpixel offset after adjustment (y, x): [0.   0.05]

------------------

1000 949
Detected subpixel offset (y, x): [ 0.   -0.01]
Detected subpixel offset after adjustment (y, x): [0.   0.05]

------------------

1000 950
Detected subpixel offset (y, x): [ 0.01 -0.01]
Detected subpixel offset after adjust

In [21]:
print(my_images_new[845])

[[ 88  95 104 ... 115 114 112]
 [ 90  92  95 ... 108 109 110]
 [ 97  89  95 ... 120 114 121]
 ...
 [106  98 101 ...  94  85  88]
 [102 109  99 ...  95  89  94]
 [102 102  98 ...  92  95  96]]


In [22]:
#plt.imshow(my_images_new[1000], cmap='gray')
my_images_new[999].shape

(1024, 1280)

# Write video

In [23]:
# import ffmpeg
# import ffprobe
# from ffprobe import FFProbe


#skvideo.setFFmpegPath("/usr/local/bin")
import skvideo
skvideo.setFFmpegPath("C:\\Users\\gt8mar\\ffmpeg-20200225-36451f9-win64-static\\bin")
import skvideo.io
import skvideo.utils
import skvideo.datasets





values_array = []

for key in my_images_new:
    values_array.append(my_images_new[key])
    
print(np.array(values_array).shape)

values_array = np.array(values_array)

skvideo.io.vwrite("20200219_174245878_patientg_2.mp4", values_array)

(1000, 1024, 1280)


In [24]:
pwd

'C:\\Users\\gt8mar\\Documents\\Marcus\\Leuk'

In [17]:
outputdata = np.random.random(size=(5, 480, 680, 3)) * 255
outputdata = outputdata.astype(np.uint8)

writer = skvideo.io.FFmpegWriter("outputvideo.mp4")
for i in xrange(5):
        writer.writeFrame(outputdata[i, :, :, :])
writer.close()

AssertionError: Cannot find installation of real FFmpeg (which comes with ffprobe).

In [24]:
width = int(1280)
height = int(1024)
FPS = len(my_images_new)
seconds = 1

In [25]:
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
video = cv2.VideoWriter('testing.avi', fourcc, FPS, (width, height))

In [28]:
sorted_my_images_new = list(sorted(my_images_new))
print(sorted_my_images_new)

for i in sorted_my_images_new:
    frame = my_images_new[i]
    #print(frame)
    video.write(frame)
video.release()

[1, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]


In [ ]:
file1 = skimage.external.tifffile.imread(file_list[0])
file2 = skimage.external.tifffile.imread(file_list[235])

In [ ]:
shift, error, diffphase = register_translation(file1, file2, 100)
print(f"Detected subpixel offset (y, x): {shift}")

In [ ]:
print(type(file1))
print(type(my_images[1]))

In [ ]:
file4 = skimage.external.tifffile.imread(file_list[3])
file4_shift = (-30.92, -45.76)

file4_offset = fourier_shift(np.fft.fftn(file4), file4_shift)
file4_offset = np.fft.ifftn(file4_offset)

shift, error, diffphase = register_translation(file2, file4_offset)
print(f"Detected subpixel offset (y, x): {shift}")

In [ ]:
type(file4_shift)

In [ ]:



shift, error, diffphase = register_translation(image, offset_image)



print(f"Detected pixel offset (y, x): {shift}")

# subpixel precision
shift, error, diffphase = register_translation(image, offset_image, 100)


# Calculate the upsampled DFT, again to show what the algorithm is doing
# behind the scenes.  Constants correspond to calculated values in routine.
# See source code for details.
cc_image = _upsampled_dft(image_product, 150, 100, (shift*100)+75).conj()
ax3.imshow(cc_image.real)
ax3.set_axis_off()
ax3.set_title("Supersampled XC sub-area")


plt.show()

print(f"Detected subpixel offset (y, x): {shift}")